In [1]:
%matplotlib inline
from vnpy.factor.backtesting.backtesting import BacktestEngine
from vnpy.factor.setting import get_backtest_data_cache_path, get_backtest_report_path


backtest_engine = BacktestEngine(
    factor_module_name="vnpy.factor.factors",
    output_data_dir_for_analyser_reports=get_backtest_report_path(),
    output_data_dir_for_calculator_cache=get_backtest_data_cache_path()
)

[vnpy.trader.setting] Updated SETTINGS from vt_setting.json
2025-06-05 05:34:14.834 | INFO | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] Successfully imported factor module: 'vnpy.factor.factors'
2025-06-05 05:34:14.835 | INFO | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] FactorMakerBacktestOrchestrator initialized.


In [2]:
vt_symbols = ['btcusdt.BINANCE', 'ethusdt.BINANCE', 'xrpusdt.BINANCE']

In [3]:
backtest_engine.output_data_dir_for_calculator_cache

PosixPath('/Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache')

In [4]:
from vnpy.factor.setting import FACTOR_DEFINITIONS_FILEPATH
from vnpy.factor.utils.factor_utils import load_factor_setting


factor_definations = load_factor_setting(FACTOR_DEFINITIONS_FILEPATH)

In [5]:
macd_factor_defination = factor_definations[2]
macd_factor_defination

{'class_name': 'MACDFactor',
 'factor_name': 'MACDFactor',
 'factor_key': 'factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26',
 'freq': '1m',
 'params': {'fast_period': 12, 'slow_period': 26, 'signal_period': 9},
 'dependencies_factor': [{'class_name': 'EMAFactor',
   'factor_name': 'fast_ema',
   'factor_key': 'factor_1m_emafactor@period_12',
   'freq': '1m',
   'params': {'period': 12},
   'dependencies_factor': [],
   'dependencies_freq': [],
   'dependencies_symbol': [],
   'dependencies_exchange': [],
   'last_run_datetime': '2025-06-04 15:35:49',
   'factor_mode': 'LIVE'},
  {'class_name': 'EMAFactor',
   'factor_name': 'slow_ema',
   'factor_key': 'factor_1m_emafactor@period_26',
   'freq': '1m',
   'params': {'period': 26},
   'dependencies_factor': [],
   'dependencies_freq': [],
   'dependencies_symbol': [],
   'dependencies_exchange': [],
   'last_run_datetime': '2025-06-04 15:35:49',
   'factor_mode': 'LIVE'}],
 'dependencies_freq': [],
 'dependencies_symbo

In [6]:
import importlib

from vnpy.factor.utils.factor_utils import init_factors


factor_module = importlib.import_module("vnpy.factor.factors")
macd_factor = init_factors(
    module_for_primary_classes=factor_module,
    settings_data=[macd_factor_defination],
    dependencies_module_lookup_for_instances=factor_module
)[0]

In [7]:
macd_factor.get_nested_params_for_optimizer()

{'fast_period': 12,
 'slow_period': 26,
 'signal_period': 9,
 'fast_ema.period': 12,
 'slow_ema.period': 26}

In [8]:
target_factor_instance, flattened_factors = backtest_engine._init_and_flatten_factor(macd_factor, vt_symbols)

2025-06-05 05:34:14.881 | INFO | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] Initializing and flattening factor based on definition. Symbols: ['btcusdt.BINANCE', 'ethusdt.BINANCE', 'xrpusdt.BINANCE']
2025-06-05 05:34:14.881 | INFO | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] Target factor instance created: factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26
2025-06-05 05:34:14.882 | DEBUG | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] Factor tree flattened. Total factors in graph: 3


In [9]:
calculator = backtest_engine._create_calculator()

2025-06-05 05:34:14.887 | INFO | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] FactorCalculator initialized. Factor cache dir: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache
2025-06-05 05:34:14.918 | INFO | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] Factor cache directory ensured at: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache


In [ ]:
from datetime import datetime

import numpy as np
import polars as pl
from typing import List, Dict

# Schema for the intermediate flat DataFrame
_OHLCV_FLAT_SCHEMA = {
    "datetime": pl.Datetime(time_unit="us", time_zone='UTC'),
    "symbol": pl.Utf8,
    "open": pl.Float64,
    "high": pl.Float64,
    "low": pl.Float64,
    "close": pl.Float64,
    "volume": pl.Float64,
}


def generate_fake_ohlcv_wide_dict(
    start_date: datetime,
    end_date: datetime,
    interval: str,
    vt_symbols: List[str]
) -> Dict[str, pl.DataFrame]:
    """
    Generates a dictionary of Polars DataFrames with fake OHLCV data.
    Each key in the dictionary is an OHLCV type ("open", "high", "low", "close", "volume").
    Each DataFrame has a "datetime" column and columns for each symbol in vt_symbols,
    containing the respective OHLCV values.

    Args:
        start_date: The start datetime for the data generation.
        end_date: The end datetime for the data generation.
        interval: Polars interval string for fake data generation (e.g., "1m", "1h").
        vt_symbols: List of symbol strings (e.g., ["BTCUSDT", "ETHUSDT"]).

    Returns:
        Dict[str, pl.DataFrame]: A dictionary where keys are OHLCV types and
                                 values are "wide" DataFrames. Returns an empty
                                 dictionary if no data can be generated.
    """
    if not vt_symbols:
        return {}

    # --- Inlined logic from _generate_fake_ohlcv_flat_df ---
    flat_df: pl.DataFrame
    try:
        fake_dates = pl.datetime_range(
            start=start_date,
            end=end_date,
            interval=interval,
            time_unit="us",  # Assuming interval is in minutes
            eager=True,
            time_zone="UTC"
        )
    except Exception: # Handle potential errors in datetime_range
        return {} # Return empty dict if date range generation fails

    if len(fake_dates) == 0:
        return {} # Return empty dict if no dates are generated

    num_fake_rows_per_symbol = len(fake_dates)
    all_rows_data = []

    for symbol in vt_symbols:
        # Generate somewhat realistic OHLCV data
        close_prices = np.random.rand(num_fake_rows_per_symbol) * 100 + 50
        open_prices = close_prices + (np.random.rand(num_fake_rows_per_symbol) - 0.5) * 10
        high_prices = np.maximum(open_prices, close_prices) + np.random.rand(num_fake_rows_per_symbol) * 5
        low_prices = np.minimum(open_prices, close_prices) - np.random.rand(num_fake_rows_per_symbol) * 5
        low_prices = np.maximum(low_prices, 0.01) # Ensure low is not zero or negative
        volumes = np.random.rand(num_fake_rows_per_symbol) * 1000 + 500

        for i in range(num_fake_rows_per_symbol):
            all_rows_data.append({
                "datetime": fake_dates[i],
                "symbol": symbol,
                "open": open_prices[i],
                "high": high_prices[i],
                "low": low_prices[i],
                "close": close_prices[i],
                "volume": volumes[i],
            })

    if not all_rows_data: # Should only happen if vt_symbols was empty, but that's checked above
        return {}

    flat_df = pl.DataFrame(all_rows_data, schema=_OHLCV_FLAT_SCHEMA)
    # --- End of inlined logic ---

    if flat_df.is_empty(): # Double check, though previous checks should cover this
        return {}

    ohlcv_dict: Dict[str, pl.DataFrame] = {}
    ohlcv_types = ["open", "high", "low", "close", "volume"]

    # Get unique sorted datetimes once for potential error case in pivot
    unique_sorted_datetimes = flat_df.get_column("datetime").unique().sort()

    for ohlcv_type in ohlcv_types:
        try:
            pivoted_df = flat_df.pivot(
                index="datetime",
                on="symbol",  # Changed "columns" to "on"
                values=ohlcv_type
            )
            # Sort by datetime just in case pivoting changes order
            pivoted_df = pivoted_df.sort("datetime")
            ohlcv_dict[ohlcv_type] = pivoted_df
        except Exception as e:
            # Handle potential errors during pivoting
            print(f"Error pivoting data for {ohlcv_type}: {e}. Creating empty structure for this type.")
            # Create an empty DataFrame with the expected structure
            empty_df_for_type = pl.DataFrame({"datetime": unique_sorted_datetimes})
            for sym in vt_symbols:
                 empty_df_for_type = empty_df_for_type.with_columns(pl.lit(None, dtype=pl.Float64).alias(sym))
            ohlcv_dict[ohlcv_type] = empty_df_for_type

    return ohlcv_dict

In [11]:
from datetime import timedelta


end_dt = datetime.now()
start_dt = end_dt - timedelta(days=5) # Generate for the last 30 minutes
time_interval = "1m" # 5-minute interval

ohlcv_data_dictionary = generate_fake_ohlcv_wide_dict(
    start_date=start_dt,
    end_date=end_dt,
    interval=time_interval,
    vt_symbols=vt_symbols
)

In [12]:
backtest_engine.memory_bar = ohlcv_data_dictionary
backtest_engine.num_data_rows = backtest_engine.memory_bar["close"].height

In [13]:
ohlcv_data_dictionary

{'open': shape: (7_201, 4)
 ┌────────────────────────────┬─────────────────┬─────────────────┬─────────────────┐
 │ datetime                   ┆ btcusdt.BINANCE ┆ ethusdt.BINANCE ┆ xrpusdt.BINANCE │
 │ ---                        ┆ ---             ┆ ---             ┆ ---             │
 │ datetime[μs]               ┆ f64             ┆ f64             ┆ f64             │
 ╞════════════════════════════╪═════════════════╪═════════════════╪═════════════════╡
 │ 2025-05-31 05:34:14.942488 ┆ 114.086702      ┆ 126.667567      ┆ 132.553282      │
 │ 2025-05-31 05:35:14.942488 ┆ 111.70094       ┆ 92.432026       ┆ 129.445144      │
 │ 2025-05-31 05:36:14.942488 ┆ 127.90584       ┆ 108.843999      ┆ 56.451906       │
 │ 2025-05-31 05:37:14.942488 ┆ 141.412078      ┆ 55.873984       ┆ 98.590973       │
 │ 2025-05-31 05:38:14.942488 ┆ 60.814211       ┆ 93.795262       ┆ 98.979267       │
 │ …                          ┆ …               ┆ …               ┆ …               │
 │ 2025-06-05 05:30:14.9424

In [14]:
factor_df = backtest_engine._run_factor_computation(
        calculator=calculator,
        target_factor_instance=target_factor_instance,
        flattened_factors=flattened_factors,
        vt_symbols_for_run=vt_symbols,  # Use the symbols for this specific run
    )

2025-06-05 05:34:15.074 | INFO | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] Starting factor value computation phase...
2025-06-05 05:34:15.075 | DEBUG | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] Aligning vt_symbols in factor 'factor_1m_emafactor@period_12' to ['btcusdt.BINANCE', 'ethusdt.BINANCE', 'xrpusdt.BINANCE']
2025-06-05 05:34:15.076 | DEBUG | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] Aligning vt_symbols in factor 'factor_1m_emafactor@period_26' to ['btcusdt.BINANCE', 'ethusdt.BINANCE', 'xrpusdt.BINANCE']
2025-06-05 05:34:15.077 | INFO | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] Starting calculation for: factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26 with symbols ['btcusdt.BINANCE', 'ethusdt.BINANCE', 'xrpusdt.BINANCE']
2025-06-05 05:34:15.078 | DEBUG | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] Determined calculation order for 3 factors.
2025-06-05 05:34:15.079 | I

In [15]:
factor_df

datetime,btcusdt.BINANCE,ethusdt.BINANCE,xrpusdt.BINANCE
"datetime[μs, UTC]",f64,f64,f64
2025-05-31 05:34:14.942488 UTC,0.0,0.0,0.0
2025-05-31 05:35:14.942488 UTC,-0.173713,-2.536002,-0.202948
2025-05-31 05:36:14.942488 UTC,0.826982,-2.799304,-4.766515
2025-05-31 05:37:14.942488 UTC,1.813659,-5.821761,-4.785565
2025-05-31 05:38:14.942488 UTC,-2.518683,-4.956467,-4.287938
…,…,…,…
2025-06-05 05:30:14.942488 UTC,1.262308,-5.159966,-1.197798
2025-06-05 05:31:14.942488 UTC,-1.270353,-2.046159,-0.085292
2025-06-05 05:32:14.942488 UTC,1.634796,-2.425114,3.543261


In [16]:
calculator.close()

2025-06-05 05:34:15.434 | INFO | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] FactorCalculator closed.
2025-06-05 05:34:15.512 | DEBUG | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] GC performed.
Factor memory file cleared and re-initialized: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache/factor_1m_emafactor@period_12.arrow
Factor memory file cleared and re-initialized: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache/factor_1m_emafactor@period_26.arrow
Factor memory file cleared and re-initialized: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache/factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26.arrow
2025-06-05 05:34:15.514 | DEBUG | FactorMakerFactorCalculator | [FactorMakerFactorCalculator] FactorMemory instances cleared.


In [17]:
market_close_prices_df = backtest_engine.memory_bar["close"].clone()

In [18]:
actual_analysis_start_dt = factor_df.select(pl.col('datetime').min()).item()
actual_analysis_end_dt = factor_df.select(pl.col('datetime').max()).item()

In [20]:
report_path = backtest_engine._run_factor_analysis(
    factor_df=factor_df,
    market_close_prices_df=market_close_prices_df,
    target_factor_instance=target_factor_instance,
    analysis_start_dt=actual_analysis_start_dt,
    analysis_end_dt=actual_analysis_end_dt,
    num_quantiles=2,
    returns_look_ahead_period=1,
    long_percentile_threshold=0.5,
    short_percentile_threshold=0.5,
    report_filename_prefix='test',
)

2025-06-05 05:37:30.884 | INFO | FactorMakerBacktestOrchestrator | [FactorMakerBacktestOrchestrator] Starting factor analysis phase...
2025-06-05 05:37:30.887 | INFO | FactorMakerFactorAnalyser | [FactorMakerFactorAnalyser] FactorAnalyser initialized. Report dir: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/reports/backtest_reports
2025-06-05 05:37:30.889 | INFO | FactorMakerFactorAnalyser | [FactorMakerFactorAnalyser] Starting analysis for factor: factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26
2025-06-05 05:37:30.890 | INFO | FactorMakerFactorAnalyser | [FactorMakerFactorAnalyser] Preparing 1-period symbol forward returns data...
2025-06-05 05:37:30.895 | INFO | FactorMakerFactorAnalyser | [FactorMakerFactorAnalyser] Symbol forward returns prepared for 3 symbols.
shape: (7_200, 4)
┌────────────────────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ datetime                       ┆ btcusdt.BINANCE ┆ ethusdt.BINANCE ┆ xrpusdt.BI

SchemaError: datatypes of join keys don't match - `datetime`: datetime[μs, UTC] on left does not match `datetime`: datetime[μs] on right

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sort' <---
DF ["datetime", "btcusdt.BINANCE", "ethusdt.BINANCE", "xrpusdt.BINANCE"]; PROJECT */4 COLUMNS

In [ ]:
market_close_prices_df = backtest_engine.memory_bar["close"].join(
    factor_df.select(pl.col("datetime")),
    on="datetime",
    how="inner",
)



market_close_prices_df.select(
    [pl.col("datetime")] + # Start with the datetime column
    [
        (
            pl.col(symbol).pct_change().shift(-1)
            .otherwise(0.0)
            .fill_null(0.0) # Handles nulls from shift(-1) or original pct_change if any
            .alias(symbol)
        )
        for symbol in backtest_engine.vt_symbols
    ]
).sort("datetime")

SchemaError: datatypes of join keys don't match - `datetime`: datetime[μs] on left does not match `datetime`: datetime[μs, UTC] on right

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'join' <---
DF ["datetime"]; PROJECT */1 COLUMNS

In [ ]:
backtest_engine.memory_bar["close"]

datetime,btcusdt.BINANCE,ethusdt.BINANCE,xrpusdt.BINANCE
datetime[μs],f64,f64,f64
2025-05-30 14:48:31.703517,75.32033,50.323127,99.757349
2025-05-30 14:49:31.703517,69.585112,123.054973,135.831781
2025-05-30 14:50:31.703517,67.198489,62.201816,124.95987
2025-05-30 14:51:31.703517,130.22661,118.451291,95.316162
2025-05-30 14:52:31.703517,57.43686,149.734169,64.163991
…,…,…,…
2025-06-04 14:44:31.703517,129.329856,70.796505,147.343227
2025-06-04 14:45:31.703517,132.032638,96.113968,85.814321
2025-06-04 14:46:31.703517,66.368801,52.728696,124.871295


In [ ]:
factor_df

datetime,btcusdt.BINANCE,ethusdt.BINANCE,xrpusdt.BINANCE
"datetime[μs, UTC]",f64,f64,f64
2025-05-30 14:48:31.703517 UTC,0.0,0.0,0.0
2025-05-30 14:49:31.703517 UTC,-0.366008,4.641576,2.302186
2025-05-30 14:50:31.703517 UTC,-0.727703,3.413422,2.9254
2025-05-30 14:51:31.703517 UTC,3.108542,6.036053,1.258957
2025-05-30 14:52:31.703517 UTC,0.700206,9.327731,-1.854376
…,…,…,…
2025-06-04 14:44:31.703517 UTC,1.45069,-1.218276,1.350531
2025-06-04 14:45:31.703517 UTC,1.855304,-1.324281,-1.237633
2025-06-04 14:46:31.703517 UTC,-2.214612,-4.113984,-0.339514
